### 국회의원 정보 스크래핑
* 이름, ID 추출
* ID를 사용해서 상세페이지로 요청 보내기
* DataFrame에 저장하고, DB에 저장
* 시각화

In [15]:
import requests
from bs4 import BeautifulSoup
import re

url = 'https://www.assembly.go.kr/assm/memact/congressman/memCond/memCondListAjax.do'
req_param = {
    #?currentPage=1&rowPerPage=500
    "currentPage":1,
    "rowPerPage":500
}

res = requests.get(url, params=req_param)
print(res.status_code)

if res.ok:
    soup = BeautifulSoup(res.text, 'html.parser')
    atag_list = soup.select("div.memberna_list dl dt a")
    
    member_list = []    
    for atag in atag_list:

        member_dict = {}
        
        member_dict['name'] = atag.text  #<a href="">둘리</a>
        
        href = atag['href']
        matched = re.search(r'(\d+)', href)
        if matched:
            member_dict['id'] = matched.group(0)
        
        #print(member_dict)
        member_list.append(member_dict)

print(len(member_list))
member_list[0:2]

200
297


[{'name': '강기윤', 'id': '9771230'}, {'name': '강대식', 'id': '9771142'}]

In [31]:
import requests
from bs4 import BeautifulSoup
import re
from urllib.parse import urljoin

print('----- 시작')
for idx,member in enumerate(member_list[0:1],1):
    print(f'---> 번호 {idx} <------')
    member_id = member['id']
    detail_url = f'https://www.assembly.go.kr/assm/memPop/memPopup.do?dept_cd={member_id}'
    
    res = requests.get(detail_url)
    if res.ok:
        soup = BeautifulSoup(res.text, 'html.parser')
        
        #1명의 국회의원 정보를 저장할 dict를 선언
        member_dict = dict()
        
        dt_list = []
        for dt in soup.select("div.info_mna dl.pro_detail dt"):
            dt_list.append(dt.text)
        
        dd_list = []
        for dd in soup.select("div.info_mna dl.pro_detail dd"):
            dd_text = dd.text.strip()
            
            #Pattern 객체생성
            reg_exr = re.compile(r'[\n\r\t]')
            #sub()는 Pattern과 일치하는 문자을 empty string으로 대체해라
            #replace()는 white space를 empty string으로 변경해라
            clean_text = reg_exr.sub('', dd_text).replace(" ","")
            dd_list.append(clean_text)
        
        print(dd_list)

----- 시작
---> 번호 1 <------
['국민의힘', '경남창원시성산구', '보건복지위원회', '재선(19대,21대)', '02-784-1751', '의원회관937호', 'http://blog.naver.com/ggotop', 'ggotop@naver.com', '김홍광,한영애', '지상훈,최광림', '김영록,안효상,이유진,홍지형,김지훈', '', '']
